In [2]:
import sys
import os

SCRIPT_DIR = os.path.dirname(os.path.abspath("..."))
sys.path.append(os.path.dirname(SCRIPT_DIR))
from training.generate import (generate_response, load_model_tokenizer_for_generate, 
                               get_special_token_id, get_special_token_id)
from training.transcript_trainer import PROMPT_FORMAT, create_data_set_from_json_list
from datasets import Dataset, DatasetDict, load_dataset, load_from_disk


In [5]:
import json
json_list_file = "/opt/home/bo_ling/dataset/eats_receipt_gcp_train.jsonl"
def gen():
    with open(json_list_file) as file:
        while True:
            line = file.readline()

            # if line is empty
            # end of file is reached
            if not line:
                break 
            try:
                data = json.loads(line)

            except:
                # print(f"BAD DATA: {line}")
                data = json.loads(line.replace("\\", ""))

            yield{
                "input_text": data['input_text'], 
                "output_text": data['output_text'].replace(" total_price", "; total_price").replace(" total_tax", "; total_tax")
            }
dataset = Dataset.from_generator(gen)
dataset = dataset.train_test_split(test_size=0.99)
dataset.save_to_disk("/opt/home/bo_ling/dataset/eats_receipt_gcp_train.hf")

Found cached dataset generator (/home/prodadmin/.cache/huggingface/datasets/generator/default-cb13d9f7f4f0a367/0.0.0)


{'uuid': 'c65024cc-26bc-490a-91d2-62f840fc0f8b',
 'field': 'dob',
 'input_text': 'Extract dob from the following input: California DRIVER LICENSE 081202 LN SOTO MUNGUIA FN ALEC PIERRE 9939 VAN RUITEN ST BELLFLOWER, CA 90706 SEX M AM DONOR USA DLY7679862 EXP 08/12/2025 DOB 08/12/2002 AGE 2 IN 7023 DD 08/04/2021606A3/E5FD/25 CLASS C END NONE RSTR NONE 08122002 HAIR BRN EYES HZL HGT 5\'-11" WGT 205 lb FEDERAL LIMITS APPLY ISS 10/14/2021',
 'output_text': '2002-08-12'}

In [4]:
train_data = load_from_disk("/opt/home/bo_ling/dataset/eats_receipt_gcp_train.hf")
train_data

DatasetDict({
    train: Dataset({
        features: ['input_text', 'output_text'],
        num_rows: 1693
    })
    test: Dataset({
        features: ['input_text', 'output_text'],
        num_rows: 2
    })
})

In [27]:
import json
json_list_file = "/opt/home/bo_ling/dataset/eats_receipt_gcp_test.jsonl"
def gen():
    with open(json_list_file) as file:
        while True:
            line = file.readline()

            # if line is empty
            # end of file is reached
            if not line:
                break 
            try:
                data = json.loads(line)

            except:
                # print(f"BAD DATA: {line}")
                data = json.loads(line.replace("\\", ""))

            yield{
                "input_text": data['input_text'], 
                "output_text": data['output_text'].replace(" total_price", "; total_price").replace(" total_tax", "; total_tax")
            }
dataset = Dataset.from_generator(gen)
dataset = dataset.train_test_split(test_size=0.99)
dataset.save_to_disk("/opt/home/bo_ling/dataset/eats_receipt_gcp_test.hf")

Generating train split: 0 examples [00:00, ? examples/s]

Dataset generator downloaded and prepared to /home/prodadmin/.cache/huggingface/datasets/generator/default-fd12e496e3ce2104/0.0.0. Subsequent calls will reuse this data.


Saving the dataset (0/1 shards):   0%|          | 0/4 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/419 [00:00<?, ? examples/s]

In [3]:
test_data = load_from_disk("/opt/home/bo_ling/dataset/eats_receipt_gcp_test.hf")
test_data

DatasetDict({
    train: Dataset({
        features: ['input_text', 'output_text'],
        num_rows: 4
    })
    test: Dataset({
        features: ['input_text', 'output_text'],
        num_rows: 419
    })
})